In [ ]:
import pandas as pd

# Load the CSV
plot_data = pd.read_csv('15MIN-Updated.csv')

plot_data.tail(5)  # shows first 20 rows


In [ ]:
import pandas as pd
import backtrader as bt

# Assuming your DataFrame is called plot_data
plot_data['DateTime'] = pd.to_datetime(plot_data['DateTime'])
plot_data.set_index('DateTime', inplace=True)
plot_data.sort_index(inplace=True)  # make sure chronological order


In [ ]:
import backtrader as bt

class MyPandasData(bt.feeds.PandasData):
    # Add new columns to the data feed
    lines = ('CrossSignal', 'SL', 'TP',)  # custom columns
    # Map dataframe column names to lines
    params = (
        ('CrossSignal', 'CrossSignal'),
        ('SL', 'SL'),
        ('TP', 'TP'),
    )


In [ ]:
data = MyPandasData(
    dataname=plot_data,
    open='Open',
    high='High',
    low='Low',
    close='Close',
    volume='Volume',
    datetime=None,  # index is DateTime
)


In [ ]:
# %% Enhanced Strategy with Report
import pandas as pd
import backtrader as bt

class CrossSignalStrategy(bt.Strategy):
    def __init__(self):
        self.cross_signal = self.datas[0].CrossSignal
        self.sl = self.datas[0].SL
        self.tp = self.datas[0].TP
        self.trades_log = []

    def next(self):
        if not self.position:
            if self.cross_signal[0] == 1:
                self.buy_bracket(
                    size=1,
                    price=self.data.open[0],
                    stopprice=self.sl[0],
                    limitprice=self.tp[0]
                )

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.last_entry_price = order.executed.price
                self.last_entry_datetime = self.data.datetime.datetime(0)

    def notify_trade(self, trade):
        if trade.isclosed:
            duration = trade.barlen  # bars held
            pnl = trade.pnl
            pnl_perc = trade.pnlcomm / self.last_entry_price * 100
            self.trades_log.append({
                'Entry Date': self.last_entry_datetime,
                'Entry Price': self.last_entry_price,
                'Exit Date': self.data.datetime.datetime(0),
                'Exit Price': trade.price,
                'PnL': pnl,
                'PnL %': pnl_perc,
                'Duration (bars)': duration,
            })

    def stop(self):
        self.trades_df = pd.DataFrame(self.trades_log)
        if self.trades_df.empty:
            print("No trades executed.")
            return

        total_trades = len(self.trades_df)
        wins = len(self.trades_df[self.trades_df['PnL'] > 0])
        losses = len(self.trades_df[self.trades_df['PnL'] <= 0])
        win_rate = wins / total_trades * 100
        total_pnl = self.trades_df['PnL'].sum()
        avg_pnl = self.trades_df['PnL'].mean()
        max_drawdown = self.trades_df['PnL'].cumsum().min()

        report = {
            'Total Trades': total_trades,
            'Winning Trades': wins,
            'Losing Trades': losses,
            'Win Rate (%)': win_rate,
            'Total PnL': total_pnl,
            'Average PnL': avg_pnl,
            'Max Drawdown': max_drawdown
        }

        self.report = report
        print("\n=== Backtest Report ===")
        for k, v in report.items():
            print(f"{k}: {v}")
        print("\nTrade Log (last 10 trades):")
        print(self.trades_df.tail(10))


In [ ]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(CrossSignalStrategy)
strat = cerebro.run()[0]

# Access report and full trade log
report = strat.report
journal = strat.trades_df
journal.to_csv("trading_journal.csv", index=False)
